In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
# Integer to Binary Generator
def getBin(integer, len_binary):
    binary = bin(int(integer))[2:].zfill(len_binary)
    return map(int, list(binary))
# Test case
number = 5.0
print getBin(number, 8)

[0, 0, 0, 0, 0, 1, 0, 1]


In [3]:
# Dataset Creation
def create_data(num_samples, len_binary):
    np.random.seed(1)
    x = np.zeros(num_samples)
    y = np.zeros((num_samples, len_binary))
    max_val = 2 ** len_binary - 1
    for i in range(num_samples):
        number = np.random.randint(0, max_val)
        x[i] = int(number)
        y[i] = getBin(number, len_binary) 
    return x, y

# Test Case
X, y = create_data(5, 4)
for i in range(X.shape[0]):
    print X[i], '\t ', y[i]

5.0 	  [ 0.  1.  0.  1.]
11.0 	  [ 1.  0.  1.  1.]
12.0 	  [ 1.  1.  0.  0.]
8.0 	  [ 1.  0.  0.  0.]
9.0 	  [ 1.  0.  0.  1.]


In [4]:
# TF Model Parameters
binary_length = 4
training_samples = 1000
testing_samples = 20
lr = 0.01
training_steps = 100000  # Need to train longer
display_steps = 5000
n_input = 1
n_hidden_units = 32  # Need more hidden units as compared to Binary to Int model
n_output = binary_length
timestep = 1

In [5]:
# Generate Training and Testing Data
X_train, y_train = create_data(training_samples, binary_length)
X_test, y_test = create_data(testing_samples, binary_length)

# Print data
display = 5
for i in range(display):
    print X_train[i], '\t', y_train[i], "\n"

5.0 	[ 0.  1.  0.  1.] 

11.0 	[ 1.  0.  1.  1.] 

12.0 	[ 1.  1.  0.  0.] 

8.0 	[ 1.  0.  0.  0.] 

9.0 	[ 1.  0.  0.  1.] 



In [6]:
# TF Model and intializations
X = tf.placeholder(tf.float32, [None, timestep, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
W = tf.Variable(tf.random_normal([n_hidden_units, n_output]))
b = tf.Variable(tf.random_normal([n_output]))

def model(X, W, b, timestep, n_hidden_units):
    X = tf.unstack(X, timestep, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)
    logits = tf.matmul(outputs[-1], W) + b
    return logits

logits = model(X, W, b, timestep, n_hidden_units)
loss = tf.reduce_mean(tf.losses.mean_squared_error(logits, y))
optimizer = tf.train.RMSPropOptimizer(lr)
training = optimizer.minimize(loss)

In [7]:
# Reshape data
X_train = np.reshape(X_train, [-1, timestep, n_input])
y_train = np.reshape(y_train, [-1, n_output])

X_test = np.reshape(X_test, [-1, timestep, n_input])
y_test = np.reshape(y_test, [-1, n_output])

# Print data
display = 5
for i in range(display):
    print X_train[i], '\t', y_train[i]

[[ 5.]] 	[ 0.  1.  0.  1.]
[[ 11.]] 	[ 1.  0.  1.  1.]
[[ 12.]] 	[ 1.  1.  0.  0.]
[[ 8.]] 	[ 1.  0.  0.  0.]
[[ 9.]] 	[ 1.  0.  0.  1.]


In [8]:
# Run TF
np.random.seed(0)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in range(training_steps):
        _, loss_out = sess.run([training, loss], feed_dict={X: X_train, y:y_train})
        if step % display_steps == 0:
            print "Loss {} at timestep {}" .format(loss_out, step)
            out = sess.run(logits, feed_dict={X: X_test})

Loss 3.79816460609 at timestep 0
Loss 0.0660181120038 at timestep 5000
Loss 0.0401009656489 at timestep 10000
Loss 0.0271296277642 at timestep 15000
Loss 0.0225992463529 at timestep 20000
Loss 0.0168229565024 at timestep 25000
Loss 0.0121213821694 at timestep 30000
Loss 0.00916683115065 at timestep 35000
Loss 0.00760125601664 at timestep 40000
Loss 0.00673454953358 at timestep 45000
Loss 0.0058130226098 at timestep 50000
Loss 0.00500587001443 at timestep 55000
Loss 0.00509011745453 at timestep 60000
Loss 0.00308739230968 at timestep 65000
Loss 0.00302260182798 at timestep 70000
Loss 0.00260396488011 at timestep 75000
Loss 0.00318238767795 at timestep 80000
Loss 0.00403110217303 at timestep 85000
Loss 0.0019497909816 at timestep 90000
Loss 0.00276785693131 at timestep 95000


In [9]:
# Evaluation Metric
# 1 if Prediction is greater than 0.5, 0 otherwise
mask = out > 0.5
out[mask] = 1
out[~mask]= 0
plot =  True
if plot is True:
    print "Ground Truth \t Predicted"
    disp = 5
    rdm = np.random.randint(0, y_test.shape[0], disp)
    for i in rdm:
        print y_test[i], "->", out[i]
acc = out == y_test
acc = acc.sum(axis=1) == binary_length
acc = acc.sum()/float(len(y_test))
print "Accuracy is {} \n" .format(acc)

Ground Truth 	 Predicted
[ 1.  1.  0.  1.] -> [ 1.  1.  0.  1.]
[ 1.  0.  0.  1.] -> [ 1.  0.  0.  1.]
[ 0.  1.  0.  1.] -> [ 0.  1.  0.  1.]
[ 1.  0.  0.  0.] -> [ 1.  0.  0.  0.]
[ 1.  0.  0.  0.] -> [ 1.  0.  0.  0.]
Accuracy is 1.0 

